# Route planner
author: Zhaoyang Ma

only use DC fast charges

assume user have access to all DCFC

charge at least once every 200 miles


## Import modules

In [38]:
import requests
import folium
import webbrowser
import json
import sys
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from geopy.distance import geodesic
from geopy.distance import distance

## Specify start and end city
input start and end

In [39]:
start = 'San Diego, CA'
end = 'San Jose, CA'

## Call MapQuest API
set generalize = 1000

apply API key from https://developer.mapquest.com/ and copy it to url1 and url2

In [40]:
# get the route overview using MapQuest API
url1 = f'http://www.mapquestapi.com/directions/v2/route?key=<YOUR_API_KEY>&from={start}&to={end}'
response1 = requests.get(url1).json()
sid = response1['route']['sessionId']
json_distance = response1['route']['distance'] # distance between start to end in miles
json_realtime = response1['route']['realTime'] # realtime of the trip
json_time = response1['route']['time'] # general time of the trip, usually < realtime

# with the sessionId obtained from above API, get the detailed route in polyline format
url2 = f'http://www.mapquestapi.com/directions/v2/routeshape?key=<YOUR_API_KEY>&sessionId={sid}&generalize=1000'
response2 = requests.get(url2).json()
lat_lng = response2['route']['shape']['shapePoints']

# record JSON files
with open('out1.json', 'w') as outfile1:
    json.dump(response1, outfile1)
outfile1.close()
with open('out2.json', 'w') as outfile2:
    json.dump(response2, outfile2)
outfile2.close()

# coordinates of the polylines turning points: coos = [lat, lng]
arr = np.array(lat_lng)
arr_2d = arr.reshape(-1, 2)
coos = arr_2d.tolist()

# add turning coordinates as [lng+lat]
coos_add = []
for c in coos:
    c1 = ('{}' +'+'+ '{}').format(c[1], +c[0])
    coos_add.append(c1)
# if less than 2 points: throw error
if len(coos_add) < 2:
    print('error')
    sys.exit(0)
# add turning coordinates together for API usage
left = coos_add[0]
for i in range(len(coos_add)-1):
    right = coos_add[i+1]
    left = ('{}' +','+ '{}').format(left, right)

## Call API from NREL.GOV website
apply API key from https://developer.nrel.gov/ and it to url3

set offset distance = 0.1 mile

In [41]:
response3 = requests.get(f"https://developer.nrel.gov/api/alt-fuel-stations/v1/nearby-route.json?api_key=<YOUR_API_KEY>&return_type=ids&fuel_type=ELEC&distance=1&route=LINESTRING({left})")
station = response3.json()

# record JSON file
with open('out3.json', 'w') as outfile3:
    json.dump(station, outfile3)
outfile3.close()

ids = station['fuel_station_ids'] # station ids
ids_str = [str(x) for x in ids] # int -> str

# compare the obtained ids with cleaned dataset and remove those not in dataset
stations = [] # stations = ['ids', 'lat', 'lng', 'street address', 'city']
with open('./dataset/new_data_cleaned2.csv', 'r') as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        if row[11] in ids_str:
            station = []
            station.append(row[11]) # ids
            station.append(row[9]) # lat
            station.append(row[10]) # lng
            station.append(row[0]) # street address
            station.append(row[1]) # city
            stations.append(station)
f.close()

## DBSCAN
eps = 10 / 6371.0088

In [42]:
# turn coordinates into Numpy type
coord = np.array([[row[1], row[2]] for row in stations])
coords = coord.astype(float)

kms_per_radian = 6371.0088
epsi = 10 / kms_per_radian
db = DBSCAN(eps=epsi, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])

print('Original stations: ', len(coord))
print('Number of clusters: ', num_clusters)

def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)
centermost_points = clusters.map(get_centermost_point)
lst = centermost_points.tolist() # lst = [lat, lon]

# use lst to filter stations
string_lst = []
for line in lst:
    line_list = []
    for l in line:
        line_list.append(str(l))
    string_lst.append(line_list)

new_stations = [] # same as stations: new_stations = ['ids', 'lat', 'lon', 'street', 'city']
for s in stations:
    s_list = []
    s_list.append(s[1])
    s_list.append(s[2])
    if s_list in string_lst:
        new_stations.append(s)

Original stations:  171
Number of clusters:  18


## Calculate route distance

In [43]:
dist_btw = [] # distance between two turning points
for i in range(len(coos)-1):
    d = 0
    pointA = coos[i]
    pointB = coos[i+1]
    d = geodesic(pointA, pointB).miles
    dist_btw.append(d)

# find stop around which coordinate to find stations
stop_count = 0 # num of charging
while ((stop_count) * 200) < json_distance: # json_distance = the real distance
    stop_count += 1
stop_seg = sum(dist_btw) / stop_count # charge after miles

dist_count = 0 # how many miles at present
stop_count2 = 0 # num of stops
coord_findstation = [] # find stations around these coordinates

for i in range(len(dist_btw)):
    dist_count += dist_btw[i]
    if dist_count > (stop_count2+1) * stop_seg:
        # find around this coordinate
        coord_findstation.append(coos[i])
        stop_count2 += 1

# calculate the closest station
closest_coordinates = []
for target in coord_findstation:
    coordinates = lst
    closest_distance = float("inf")
    closest_coordinate = None

    for coor in coordinates:
        dist = distance(target, coor).miles  # calculate distance in miles
        if dist < closest_distance:
            closest_distance = dist
            closest_coordinate = coor
    closest_coordinates.append(closest_coordinate)

# turn closest_coordinates into list & str
closest_coordinates_strs = [] # closest_coordinates_strs = ['lat', 'lon']
for i in range(len(closest_coordinates)):
    closest_coordinates_str = []
    closest_coordinates_str.append(str(closest_coordinates[i][0]))
    closest_coordinates_str.append(str(closest_coordinates[i][1]))
    closest_coordinates_strs.append(closest_coordinates_str)


## Plot map with folium

In [44]:
# initialize the map
m = folium.Map(location=[38, -120], zoom_start=6)

# plot route with polylines
folium.PolyLine(coos, tooltip="Route").add_to(m)

# mark the new stations
for i in range(len(new_stations)):
    test_list = []
    test_list.append(new_stations[i][1])
    test_list.append(new_stations[i][2])

    if test_list in closest_coordinates_strs:
        # recommended station with red
        folium.Marker(
            [new_stations[i][1],new_stations[i][2]],
            icon=folium.Icon(color="red",icon="plus-sign"),
            popup = new_stations[i][3] +' '+ new_stations[i][4]
        ).add_to(m)
    else:
        # other stations
        folium.Marker(
            [new_stations[i][1],new_stations[i][2]],
            popup = new_stations[i][3] +' '+ new_stations[i][4]
        ).add_to(m)

m